I tried to just load the mhd files directly before doing any preprocessing, but then the kernel died after 21 images loaded, so I might have to do some preprocessing directly after loading the image and then saving that smaller array. When printing the shape of the ct scans I could also see that each slize was 512*512 but that a different number of slices was used for each ct scan, from 100 to 500, so it could differ quite a lot. I don´t know if the slice number has a certain significance. Looking at this article: https://soundvet.com/portable-ct/understanding-ct-scanner-specifications-slice-count-scan-speed-and-cost-efficiency/ it seems like a whole ct scanner might be set to a certain number of slices, but maybe there are scanners where you can adjust the number of slices? According to this article: https://lbnmedical.com/ct-scan-slice-types/ a CT scanner takes 4 - 640 slices. According to the article it seems like our ct scans with over 100 slices are very detailed, have very high image quality and have been taken with expensive machines - alright. I´m still a bit confused about the irregular number of slices, I would expect powers of 2, maybe 128,256 etc but I´m seeing 204, 157, 139 etc - maybe I´ll figure that one out later. 

Since we only need 20 images anyways for the training, maybe we can load just 20 imaged and train the model in this notebook. Then we can load the model from another notebook and do the validation there. 

In [1]:
#SimpleITK helps us load the mhd files into numpy arrays

import SimpleITK as sitk
import numpy as np
import glob
import os

local_path = '/home/ec2-user/SageMaker/images/'

filenames_train = []
for idx,filename in enumerate(glob.glob(os.path.join(local_path, '*.mhd'))):
    filenames_train.append(filename)
    if(idx == 19):
        break

print(len(filenames_train))

20


In [3]:
def load_mhd_file(file_path):
    # Read the MHD file using SimpleITK
    image = sitk.ReadImage(file_path)
    
    # Convert it to a NumPy array (H x W x D)
    image_array = sitk.GetArrayFromImage(image)
    
    return image_array

image_arrays = []
for filename in filenames_train:
    image_data = load_mhd_file(filename)
    image_arrays.append(image_data)

I found this tutorial for finetuning a U-net model on SageMaker Neo: https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_neo_compilation_jobs/tensorflow_unet/sagemaker-neo-tf-unet.html

I might need to do some changes to change it to using a 3D U-net model though. 